In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/custom_faster_rcnn.py')

root='/data/ephemeral/home/dataset/'

cfg.keys()

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'opencv_num_threads', 'mp_start_method', 'auto_scale_lr'])

In [3]:
cfg.data

{'samples_per_gpu': 2,
 'workers_per_gpu': 2,
 'train': {'type': 'CocoDataset',
  'ann_file': 'data/coco/annotations/instances_train2017.json',
  'img_prefix': 'data/coco/train2017/',
  'pipeline': [{'type': 'LoadImageFromFile'},
   {'type': 'LoadAnnotations', 'with_bbox': True},
   {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
   {'type': 'RandomFlip', 'flip_ratio': 0.5},
   {'type': 'Normalize',
    'mean': [123.675, 116.28, 103.53],
    'std': [58.395, 57.12, 57.375],
    'to_rgb': True},
   {'type': 'Pad', 'size_divisor': 32},
   {'type': 'DefaultFormatBundle'},
   {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}]},
 'val': {'type': 'CocoDataset',
  'ann_file': 'data/coco/annotations/instances_val2017.json',
  'img_prefix': 'data/coco/val2017/',
  'pipeline': [{'type': 'LoadImageFromFile'},
   {'type': 'MultiScaleFlipAug',
    'img_scale': (1333, 800),
    'flip': False,
    'transforms': [{'type': 'Resize', 'keep_ratio': True},
     {'type': 'Rand

In [4]:
cfg.model

{'type': 'FasterRCNN',
 'backbone': {'type': 'mmcls.ConvNeXt',
  'arch': 'base',
  'out_indices': (0, 1, 2, 3),
  'drop_path_rate': 0.2,
  'layer_scale_init_value': 1.0,
  'gap_before_final_norm': False,
  'init_cfg': {'type': 'Pretrained',
   'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/convnext/convnext-base_3rdparty_in1k_20220124-dcc16c5d.pth'}},
 'neck': {'type': 'FPN',
  'in_channels': [128, 256, 512, 1024],
  'out_channels': 256,
  'num_outs': 5},
 'rpn_head': {'type': 'RPNHead',
  'in_channels': 256,
  'feat_channels': 256,
  'anchor_generator': {'type': 'AnchorGenerator',
   'scales': [8],
   'ratios': [0.5, 1.0, 2.0],
   'strides': [4, 8, 16, 32, 64]},
  'bbox_coder': {'type': 'DeltaXYWHBBoxCoder',
   'target_means': [0.0, 0.0, 0.0, 0.0],
   'target_stds': [1.0, 1.0, 1.0, 1.0]},
  'loss_cls': {'type': 'CrossEntropyLoss',
   'use_sigmoid': True,
   'loss_weight': 1.0},
  'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}},
 'roi_head': {'type': 'StandardRo

In [6]:
cfg.checkpoint_config

{'interval': 1}

In [6]:
cfg.evaluation

{'interval': 1, 'metric': 'bbox'}

In [7]:
cfg.log_config = dict(
    interval=50,  # 로그를 50 iteration마다 출력
    hooks=[
        dict(type='TextLoggerHook'),  # 기본 Text Logger
        dict(type='TensorboardLoggerHook'),  # Tensorboard Logger 추가
    ])

In [9]:
# dataset config 수정

cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

'''
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train_split.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'val_split.json' # train json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize
'''

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 8 # 한 GPU가 학습 시 한 번에 처리할 이미지(샘플)의 개수(배치 크기 조절)
cfg.data.workers_per_gpu = 4 # 한 GPU가 데이터를 불러올 때 병렬로 작업할 워커(worker)의 수

cfg.seed = 2024
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/custom_faster_rcnn_fpn_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2) # 학습 중 기울기 폭발을 방지하기 위해 사용
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1) # 최대 3개의 체크포인트 파일을 유지

cfg.evaluation = dict(
    interval=1,
    metric='bbox',
    by_epoch=True
)

cfg.device = get_device()

In [10]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


In [11]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [12]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

AttributeError: FasterRCNN: 'NoneType' object has no attribute 'parent'

In [12]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-10-14 13:49:09,955 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-14 13:49:09,958 - mmdet - INFO - Start running, host: root@instance-12208, work_dir: /data/ephemeral/home/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash
2024-10-14 13:49:09,959 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      

KeyboardInterrupt: 